In [1]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
import tensorflow_model_optimization as tfmot

In [15]:
# Load pretrained model (best saved one)
with open('./model/finalproject_model.json', 'r') as json_file:
    json_savedModel= json_file.read()
    
# load the model  
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights('./model/finalproject_weights.h5')
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.0001),
    metrics= ["accuracy"]
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                             

In [16]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3

In [17]:
 model.layers

In [21]:
# Separate the base_model from the rest of the model
base_model = model.layers[1]
rest_of_model = model.layers[2:]

# Create a quantization-aware model for the rest of the model
quantize_model = tfmot.quantization.keras.quantize_model

# Manually build the model or infer the input shape automatically
rest_of_model = tf.keras.Sequential(rest_of_model)
rest_of_model.build(input_shape=(None, 8, 8, 2048))

quantized_rest_of_model = quantize_model(rest_of_model)

In [23]:
# Combine the quantized rest_of_model with the base_model
quantized_model = tf.keras.Sequential([model.layers[0], base_model, quantized_rest_of_model])

quantized_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["accuracy"]
)

quantized_model.build(input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS))
quantized_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 sequential_13 (Sequential)  (None, 4)                 1213856   
                                                                 
Total params: 24,801,568
Trainable params: 1,213,828
Non-trainable params: 23,587,740
_________________________________________________________________


In [24]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../datasets-4",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

total_batches = len(dataset)
train_size = int(0.75 * total_batches)
test_size = int(0.10 * total_batches)
val_size = total_batches - train_size - test_size

print(f"""
    Total Batches: {total_batches},
    Training Batches: {train_size},
    Testing Batches: {test_size},
    Validation Batches: {val_size}
""")

train_ds = dataset.take(train_size) # used for training data
test_ds = dataset.skip(train_size).take(test_size) # final evaluation will be done on this model
val_ds = dataset.skip(train_size + test_size).take(val_size) # used to tune the model

Found 2814 files belonging to 4 classes.

    Total Batches: 88,
    Training Batches: 66,
    Testing Batches: 8,
    Validation Batches: 14



In [25]:
train_ds = train_ds.cache().shuffle(2000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(2000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(2000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [26]:
quantized_model.fit(train_ds, epochs=10)

Epoch 1/10


2023-05-19 09:47:34.264523: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 24 of 2000
2023-05-19 09:47:44.398488: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 58 of 2000
2023-05-19 09:47:47.325555: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2023-05-19 09:47:48.441974: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2023-05-19 09:47:49.694945: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 1/66 [..............................] - ETA: 33:26 - loss: 0.1154 - accuracy: 0.9688

2023-05-19 09:47:52.394671: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-05-19 09:47:52.394708: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-05-19 09:47:52.447569: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-05-19 09:47:52.447599: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Alloc

66/66 [==============================] - 44s 206ms/step - loss: 0.1401 - accuracy: 0.9432
Epoch 2/10
66/66 [==============================] - 14s 207ms/step - loss: 0.1323 - accuracy: 0.9451
Epoch 3/10
66/66 [==============================] - 14s 207ms/step - loss: 0.1224 - accuracy: 0.9569
Epoch 4/10
66/66 [==============================] - 14s 207ms/step - loss: 0.1089 - accuracy: 0.9593
Epoch 5/10
66/66 [==============================] - 14s 207ms/step - loss: 0.1133 - accuracy: 0.9574
Epoch 6/10
66/66 [==============================] - 14s 207ms/step - loss: 0.1107 - accuracy: 0.9635
Epoch 7/10
66/66 [==============================] - 14s 207ms/step - loss: 0.1042 - accuracy: 0.9574
Epoch 8/10
66/66 [==============================] - 14s 207ms/step - loss: 0.1026 - accuracy: 0.9621
Epoch 9/10
66/66 [==============================] - 14s 207ms/step - loss: 0.0924 - accuracy: 0.9635
Epoch 10/10
66/66 [==============================] - 14s 208ms/step - loss: 0.0849 - accuracy: 0.9697


In [27]:
scores = quantized_model.evaluate(test_ds)

2023-05-19 09:54:21.476393: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 1 of 2000
2023-05-19 09:54:21.755415: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 2 of 2000
2023-05-19 09:54:23.889349: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.


8/8 [==============================] - 29s 212ms/step - loss: 0.1112 - accuracy: 0.9648


In [28]:
# Convert the quantized model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized TFLite model to a file
with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

2023-05-19 09:55:52.563207: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpe5yls_q9/assets


INFO:tensorflow:Assets written to: /tmp/tmpe5yls_q9/assets
/home/totoro/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-19 09:56:12.002780: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-05-19 09:56:12.002812: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-05-19 09:56:12.007560: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpe5yls_q9
2023-05-19 09:56:12.042761: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-05-19 09:56:12.042797: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpe5yls_q9
2023-05-19 09:56:12.171055: I tensorflow/cc/saved_model/loader.cc: